In [1]:
import numpy as np

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, function):
        self.creator = function

In [3]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # make output remember who create itself
        self.input = input
        self.output = output  # remember output
        return output
    
    def forward(self, x):
        raise NotImplementedError


In [7]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [8]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(.5))
a = A(x)
b = B(a)
y = C(b)

In [9]:
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [10]:
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [11]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [12]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# add backward in Variable

In [13]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, function):
        self.creator = function
        
    def backward(self):
        function = self.creator
        if function is not None:
            x = function.input
            x.grad = function.backward(self.grad)
            x.backward()

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(.5))
a = A(x)
b = B(a)
y = C(b)

# backpropagation
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# apply loop on backward

In [15]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [16]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
